In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
test = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv")
embeding_file_path = "../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec"

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
max_features = 100000
embed_size = 300
maxlen = 100

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(pd.concat((train['comment_text'],test['comment_text'])))
train_words = tokenizer.texts_to_sequences(train['comment_text'])
test_words = tokenizer.texts_to_sequences(test['comment_text'])
train_words = pad_sequences(train_words, maxlen=maxlen,padding="post", truncating="post")
test_words = pad_sequences(test_words, maxlen=maxlen, padding="post", truncating="post")

In [ ]:
train_words[1099]

In [ ]:
def get_coef(word, *coefs):
    return word, np.asarray(coefs, dtype=np.float64)
embeding_dict = dict(get_coef(*s.strip().split(" ")) for s in open(embeding_file_path))

In [ ]:
word_index = tokenizer.word_index
max_words = min(max_features, len(word_index))
embeding_matrix = np.random.randn(max_words+1, embed_size)
for word,i in word_index.items():
    if word not in embeding_dict: continue
    if i>max_words:continue 
    embeding_matrix[i] = embeding_dict[word]


In [ ]:
len(embeding_dict)

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import Conv1D, MaxPooling1D, Reshape, Flatten, Dropout
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

num_filters = 100
filter_sizes = [2,4,5]

def get_model():
    inp = Input(shape=(maxlen,)) #maxlen
    x = Embedding(max_words+1, embed_size, weights=[embeding_matrix])(inp)
    x = Dropout(0.5)(x)
    conv_0 = Conv1D(num_filters, filter_sizes[0], activation='relu', init='he_normal', padding='valid', strides=1)(x)
    conv_1 = Conv1D(num_filters, filter_sizes[1], activation='relu', init='he_normal', padding='valid', strides=1)(x)
    conv_2 = Conv1D(num_filters, filter_sizes[2], activation='relu', init='he_normal', padding='valid', strides=1)(x)
    max_pool_0 = MaxPooling1D(2)(conv_0)
    max_pool_0 = Flatten()(max_pool_0)
    max_pool_1 = MaxPooling1D(2)(conv_1)
    max_pool_1 = Flatten()(max_pool_1)
    max_pool_2 = MaxPooling1D(2)(conv_2)
    max_pool_2 = Flatten()(max_pool_2)
    conc = concatenate([max_pool_0, max_pool_1, max_pool_2])
    conc = Dropout(0.3)(conc)
    fc = Dense(50, activation='relu')(conc)
    oup = Dense(6, activation='sigmoid')(fc)
    
    model = Model(input=inp, output=oup)
    model.compile(loss='binary_crossentropy', optimizer='Adadelta', metrics=['accuracy'])
    return model

model = get_model()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

batch_size = 128
epochs = 10
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_tra, X_val, y_tra, y_val = train_test_split(train_words, y_train, train_size=0.9, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)
hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc, early_stop], verbose=1)
y_pred = model.predict(test_words, batch_size=1024)
          

In [ ]:
model.save("text_cnn_v2.h5")

In [ ]:
submission = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('text_cnn_v2.csv', index=False)           

In [ ]:
submission